# Answering Business Questions Using SQL

Some more practice in SQLite with `chinook.db`, a database from the Chinook record store, sourced from [here](https://github.com/lerocha/chinook-database), attempting to answer some realistic business questions.

Database schema ([license](./LICENSE.md)):

![Chinook database schema image](./chinook-schema.svg)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

* sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Picking New Albums to Stock

Scenario: a deal has been signed with a record label, and 3 new albums out of a potential list of 4 are to be added to Chinook's store. The label specialises in artists from the USA. To determine which ones to pick, an understanding of which genres sell best in the USA should be developed. More precisely, how many tracks have been sold in the USA, per genre, in absolute and percentage numbers?  

The available artists are:

| Artist Name           | Genre     |
| --------------------- | --------- |
| Regal                 | Hip-Hop   |
| Red Tone              | Punk      |
| Meteor and the Girls  | Pop       |
| Slim Jim Bites        | Blues     |

To build up a query to answer this question, the following steps will be performed:

1. Create a temporary view of `customer` querying rows for customers based in the USA.
2. Build on this view by joining `invoice`, `invoice_line`, `track`, and `genre` with LEFT JOINs respectively.
3. Query the above temporary view, aggregating on genre.

In [40]:
%%sql
  WITH usa_customers AS (
       SELECT *
         FROM customer
        WHERE country = 'USA'
       ),
       usa_tracks AS (
       SELECT (uc.first_name || " " || uc.last_name) AS customer_name,
              uc.country,
              i.invoice_id,
              il.invoice_line_id,
              t.track_id,
              t.name AS track_name,
              g.name AS genre
         FROM usa_customers AS uc
              LEFT JOIN invoice AS i
              ON uc.customer_id = i.customer_id
              LEFT JOIN invoice_line AS il
              ON i.invoice_id = il.invoice_id
              LEFT JOIN track AS t
              ON il.track_id = t.track_id
              LEFT JOIN genre AS g
              ON t.genre_id = g.genre_id
       )

SELECT genre,
       COUNT(*) AS tracks_sold_absolute,
       ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM usa_tracks) * 100, 1) AS tracks_sold_perc
  FROM usa_tracks
 GROUP BY genre
 ORDER BY tracks_sold_absolute DESC
 LIMIT 10;

* sqlite:///chinook.db
Done.


genre,tracks_sold_absolute,tracks_sold_perc
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


The above query provides the top 10 genres of tracks sold in the USA. Of the four artists available, the above query suggests the following preference for expected sales performance:

1. Red Tone (Punk)
2. Slim Jim Bites (Blues)
3. Meteor and the Girls (Pop)
4. Regal (Hip Hop)

However, there is only 0.2% difference in track sales between Pop and Hip Hop, so either Meteor and the Girls or Regal could be selected as a 3rd preference.

In addition, it should be noted that the genres of the new albums make up a total of 20% the revenue at Chinook. New Rock albums would likely be more lucrative should they be made available.

## Analysing Purchases Assigned to Sales Support Agents

The below query finds the total sales amount for each sales support agent employed at Chinook.

In [45]:
%%sql
  WITH sales_rep_data AS (
       SELECT c.customer_id,
              c.support_rep_id,
              i.invoice_id,
              i.total,
              e.*
         FROM customer AS c
              LEFT JOIN invoice AS i
              ON c.customer_id = i.customer_id
              LEFT JOIN employee AS e
              ON c.support_rep_id = e.employee_id
       )
SELECT ROUND(SUM(srd.total), 2) AS total_sales,
       (srd.first_name || " " || srd.last_name) AS employee_name,
       srd.title,
       srd.hire_date,
       (e.first_name || " " || e.last_name) AS manager
  FROM sales_rep_data AS srd
       LEFT JOIN employee AS e
       ON srd.reports_to = e.employee_id
 GROUP BY srd.employee_id
 ORDER BY total_sales DESC;

* sqlite:///chinook.db
Done.


total_sales,employee_name,title,hire_date,manager
1731.51,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,Nancy Edwards
1584.0,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,Nancy Edwards
1393.92,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,Nancy Edwards


There is a 20% range in total sales for the sales support agents. This is not an indicator of relative performance though, because the query did not set boundaries for the invoice data with which to compare one agent to another, such as a specific time period. Looking at the query result, the total sales figure correlates with hire date, i.e. the agents who have been working at Chinook longer have higher total sales figures.

## Country Sales data (Attempt \#1)

_This 1st attempt section is left here as evidence of my learning process. I struggled at first with how to calculate average order value per country and calculate average order value per customer per country. The steps below were my attempt to compartmentalise the process for generating the query, but ultimately led to a rather complicated piece of SQLite code that didn't quite get the result I needed._

_I had a peek at the solution notebook to guide me towards a simpler and more elegant structure for the query._

Data desired per country, with countries with only 1 customer grouped into an 'Other' category:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Steps to produce the query:

1. Create temporary view `sales_agg`, grouped by `customer.country`, showing country, total order value, and average order value (per country).
2. Create temporary view `cust_agg`, grouped by `customer.customer_id`, showing country, customer ID, and average order value (per customer).
3. Create temporary view `cust_country_agg` from (2), grouped on `customer.country`, showing country, customer ID, and average order value (now per customer per country).
4. Combine `sales_agg` and `cust_country_agg` in temporary view `combined`, adding the following columns:
    - `country_sorting`: values are 'Other' if only 1 customer from that country, otherwise the country name.
    - `other_sort`: 1 if only 1 customer from that country, otherwise 0; this allows sorting with 'Other' below all named countries.
5. Create temporary view `combined_other_agg` from `combined`, which groups the data by `combined.country_sorting` and provides some formatting for final data.
6. Finally, query `combined_other_agg`, selecting desired columns and ordering by `other_sort` ascending.

In [79]:
%%sql
  WITH sales_agg AS(
       SELECT c.country,
              SUM(i.total) AS sales_tot,
              AVG(i.total) AS avg_order_val
         FROM customer AS c
              LEFT JOIN invoice AS i
              ON c.customer_id = i.customer_id
        GROUP BY c.country
       ),

       cust_agg AS(
       SELECT c.country,
              c.customer_id,
              AVG(i.total) AS avg_tot
         FROM customer AS c
              LEFT JOIN invoice AS i
              ON c.customer_id = i.customer_id
        GROUP BY c.customer_id
       ),

       cust_country_agg AS(
       SELECT country,
              COUNT(customer_id) AS num_cust,
              AVG(avg_tot) AS avg_val_per_cust
         FROM cust_agg
        GROUP BY country
       ),

       combined AS(
       SELECT *,
              CASE
                  WHEN c.num_cust = 1 THEN 'Other'
                  ELSE c.country
              END AS country_sorting,
              CASE
                  WHEN c.num_cust = 1 THEN 1
                  ELSE 0
              END AS other_sort
         FROM sales_agg AS s
              LEFT JOIN cust_country_agg AS c
              ON s.country = c.country
       ),

       combined_other_agg AS(
       SELECT country_sorting AS country,
              ROUND(SUM(sales_tot), 2) AS sales_total,
              ROUND(AVG(avg_order_val), 2) AS avg_order_value,
              SUM(num_cust) AS num_customers,
              ROUND(AVG(avg_val_per_cust), 2) AS avg_value_per_customer,
              other_sort
         FROM combined
        GROUP BY country_sorting
        ORDER BY sales_total DESC
       )
       
SELECT country,
       num_customers,
       sales_total,
       avg_order_value,
       avg_value_per_customer
  FROM combined_other_agg
 ORDER BY other_sort ASC;

* sqlite:///chinook.db
Done.


country,num_customers,sales_total,avg_order_value,avg_value_per_customer
USA,13,1040.49,7.94,8.01
Canada,8,535.59,7.05,7.42
Brazil,5,427.68,7.01,6.99
France,5,389.07,7.78,7.84
Germany,4,334.62,8.16,8.19
Czech Republic,2,273.24,9.11,9.38
United Kingdom,3,245.52,8.77,8.75
Portugal,2,185.13,6.38,6.53
India,2,183.15,8.72,8.76
Other,15,1094.94,7.45,7.45


## Country Sales data (Attempt \#2)

_This is a much more straightforward query setup, it is ultimately easier to follow and has less potential for error. One main piece of code I took from the solution notebook was:_

```
CASE
    WHEN
        (
        SELECT COUNT(*)
            FROM customer
        WHERE country = c.country
        ) = 1 THEN 'Other'
    ELSE c.country
END AS country,
```

_This counts the number of rows in _`customer`_ for each country; if there is only 1 customer, then it returns 'Other', otherwise the country name. I found it to be a very elegant method of creating a view where the country column replace single-customer country names with 'Other'._

_The other thing I picked up on was the use of _`DISTINCT`_. I was going around in circles in my head about grouping a table on countries, but then each country could still have multiple instances of customers, for those who would have made multiple orders. It was a simple command that I had momentarily forgotten._

_With the above two points, the query below is remarkably close to the solution notebook. However, my mind works better with compartmentalised sections of code, so where my second temporary view _`agg_data`_ can fit as a sub-query, my preference was not to do this._

Data desired per country, with countries with only 1 customer grouped into an 'Other' category:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Steps to produce query:

1. Create temporary view `country_sort`, joining `customer` and `invoice`, and replacing country names with just 1 customer as 'Other'.
2. Create temporary view `agg_data`, creating a `sort` column so that 'Other' country data can be aggregated below the rest. The desired data is calculated, and sorted on `sort` ascending followed by `sales_total` descending.
3. Main query selects all columns from `agg_data` except for `sort`.

In [101]:
%%sql

  WITH country_sort AS
       (
       SELECT CASE
                  WHEN 
                       (
                       SELECT COUNT(*)
                         FROM customer
                        WHERE country = c.country
                       ) = 1 THEN 'Other'
                  ELSE c.country
              END AS country,
              c.customer_id,
              i.invoice_id,
              i.total
         FROM customer AS c
              LEFT JOIN invoice AS i
              ON c.customer_id = i.customer_id
       ),

       agg_data AS
       (
       SELECT CASE
                  WHEN country = 'Other' THEN 1
                  ELSE 0
              END AS sort,
              country,
              COUNT(DISTINCT customer_id) AS customers,
              ROUND(SUM(total), 2) AS sales_total,
              ROUND(SUM(total) / COUNT(DISTINCT customer_id), 2) AS value_per_customer,
              ROUND(AVG(total), 2) AS avg_order_value
         FROM country_sort
        GROUP BY country
        ORDER BY sort ASC, sales_total DESC
       )

SELECT country,
       customers,
       sales_total,
       value_per_customer,
       avg_order_value
  FROM agg_data;

* sqlite:///chinook.db
Done.


country,customers,sales_total,value_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


## Whole Album vs Individual Track Sales

Chinook's customer purchasing system either allows an album to be purchased in a single order, or a selection of tracks to be selected for an order, but does not allow an entire album to be selected followed by individually selected tracks in an order (unless all individual tracks in an album are selected).

Chinook would like to understand the proportion of purchases of whole albums compared to purchases with individual tracks. This is with a view to potentially update the purchasing strategy to only buy popular tracks from albums rather than full albums.

Edge cases and their assumptions for this analysis:

1. Purchases of albums with just 1 or 2 tracks; since these albums would be purchased as a whole in either scenario, this can be ignored.
2. Purchases of an album (tracks individually selected) with other tracks added to the order; it is assumed that Chinook has determined that these kinds of orders represent a negligible proportion of total orders.

To build this query, the following steps are taken:

1. Temporary view `album_lookup`, which joins `invoice_line` and `track` on 'track_id', grouping by 'invoice_id'. This selects an album ID for each invoice; there is no need to be selective on which album ID within a given invoice.
2. Temporary view `album_tracks`, which joins `album_lookup` and `track` on `album_id`, generating a set of album tracks for each invoice ID. This is to later provide a comparison to see if a given invoice was to purchase a whole album or a set of individual tracks.
3. Temporary view `invoice_tracks`, which joins `invoice_line` and `track` on 'track_id' without any grouping. This provides the list of tracks for each invoice.
4. Temporary view `classification`, which compares (2) and (3) using an EXCEPT clause, to classify each invoice as 'album' or 'tracks'
5. Main query pulls from (4) the purchase type (album or tracks), and calculates for each type: the number of invoices, the percentage of invoices, and the total associated revenue.

In [137]:
%%sql

  WITH album_lookup AS
       (
       SELECT il.invoice_id,
              t.album_id,
              t.track_id
         FROM invoice_line AS il
              LEFT JOIN track AS t
              ON il.track_id = t.track_id
        GROUP BY invoice_id
       ),

       album_tracks AS
       (
       SELECT al.invoice_id,
              al.album_id,
              t.track_id
         FROM album_lookup AS al
              LEFT JOIN track AS t
              ON al.album_id = t.album_id
       ),

       invoice_tracks AS
       (
       SELECT il.invoice_id,
              t.album_id,
              t.track_id
         FROM invoice_line AS il
              LEFT JOIN track AS t
              ON il.track_id = t.track_id
       ),

       classification AS
       (
       SELECT invoice_id,
              CASE
                  WHEN (
                  SELECT track_id FROM album_tracks WHERE invoice_id = il.invoice_id
                  EXCEPT
                  SELECT track_id FROM invoice_tracks WHERE invoice_id = il.invoice_id
                  ) IS NULL THEN 'album'
                  ELSE 'tracks'
              END AS purchase_type
         FROM invoice_line AS il
        GROUP BY il.invoice_id
       )

SELECT c.purchase_type,
       COUNT(c.purchase_type) AS sales_count,
       ROUND(CAST(COUNT(c.purchase_type) AS FLOAT) / (SELECT COUNT(*) FROM invoice) * 100, 1) AS sales_perc,
       ROUND(TOTAL(i.total), 2) AS sales_revenue
  FROM classification AS c
       LEFT JOIN invoice AS i
       ON c.invoice_id = i.invoice_id
 GROUP BY purchase_type;

* sqlite:///chinook.db
Done.


purchase_type,sales_count,sales_perc,sales_revenue
album,130,21.2,1575.09
tracks,484,78.8,3134.34


Just over 20% of orders (invoices) are for albums, but these account for around a third of store revenue. One crucial dimension that has not been assessed (because the data is not in the database) is the cost to Chinook for purchasing tracks/albums from labels for onward retail. If the store decides to only purchase a couple of popular tracks from albums instead of the full albums, then the cost of buying from labels reduces. However, the potential revenue from full album sales also reduces, because the margins from the less popular tracks are no longer available (assuming all tracks have a positive margin associated with them). Without assessing how this could balance out, it is difficult to make a recommendation. There is also a risk that some 'album' sales may not convert to 'tracks' sales for customers who prefer to purchase full albums; without stocking full albums these customers may be lost as a result of the proposed purchasing strategy.